# V19 (gated 82+83): Alias diagnostics

Re-runs harmonic comparisons and phase-shift event searches on the gated (82+83) dataset.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

sectors_used = [82, 83]
ds = sh.load_dataset()
lc = sh.stitch_pdcsap_sectors(ds, sectors_used)
depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False, preset='extended')
r = session.run('V19')

print(json.dumps({
  'sectors_used': sectors_used,
  'depth_ppm': depth_ppm,
  'depth_err_ppm': depth_err_ppm,
  'status': r.status,
  'flags': list(r.flags),
  'metrics': r.metrics,
}, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "depth_err_ppm": 11.153641839855947,
  "depth_ppm": 264.57118442002604,
  "flags": [],
  "metrics": {
    "base_depth_ppm_P": 266.4183853079427,
    "base_score_P": 27.803016918031517,
    "best_other_depth_ppm": 199.05265531505734,
    "best_other_harmonic": "2P",
    "best_other_over_base_score_ratio": 0.631055761510029,
    "best_other_score": 17.5452540134846,
    "max_phase_shift_event_sigma": 10.307576605399433,
    "n_phase_shift_events": 4,
    "secondary_significance_sigma": 0.0
  },
  "sectors_used": [
    82,
    83
  ],
  "status": "ok"
}
```

</details>


In [ ]:
import json
import matplotlib.pyplot as plt

from tess_vetter.plotting import plot_alias_diagnostics

step_id = '28_gated_82_83_v19_alias_diagnostics'
fname = 'V19_alias_diagnostics.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

out = {'status': r.status, 'flags': list(r.flags)}

if r.status == 'ok':
    fig, ax = plt.subplots(figsize=(9, 5))
    plot_alias_diagnostics(r, ax=ax)
    ax.set_title('V19 (gated 82+83): Alias diagnostics')
    fig.tight_layout()
    fig.savefig(run_path, dpi=160, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=160, bbox_inches='tight')
    plt.show()
    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/28_gated_82_83_v19_alias_diagnostics/V19_alias_diagnostics.png",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/28_gated_82_83_v19_alias_diagnostics/V19_alias_diagnostics.png",
  "status": "ok"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/28_gated_82_83_v19_alias_diagnostics/V19_alias_diagnostics.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** the strongest competing harmonic is still `2P` (score ratio ≈ 0.63); phase-shift events remain (n=4, max ≈ 10.3σ).
- **Why this is useful:** helps diagnose whether the period is competing strongly with harmonics/aliases and whether structured events exist away from phase 0.
- **Interpretation:** treat as ongoing caution; combine with stellar-variability diagnostics (V12) and re-check odd/even and secondary on the gated dataset.
- **Next step:** re-run V11–V12 on the gated dataset to see whether variability signatures line up with these warnings.

</details>
